# Load data

In [1]:
import pandas as pd
path_to_data = '/Users/dosmirnov/Downloads/news.txt'

df = pd.read_csv(path_to_data, sep='\t', names = ('labels', 'header', 'text'))
df.head()

,labels,header,text
0,style,Rolex наградит победителей регаты,Парусная гонка Giraglia Rolex Cup пройдет в Ср...
1,sport,Матс Сундин стал советником тренера сборной Шв...,Шведский хоккеист Матс Сундин назначен советни...
2,media,Брендом года по версии EFFIE впервые стал город,"Гран-при конкурса ""Брэнд года/EFFIE"" получил г..."
3,economics,Цена нефти WTI снизилась после публикации данн...,Цена американской нефти WTI на лондонской бирж...
4,economics,Сбербанк распродаст другим банкирам миллиардны...,"Сбербанк выставил на продажу долги по 21,4 тыс..."


In [2]:
from dataclasses import dataclass
from typing import Optional


@dataclass
class Entry:
    name: str
    birth_date: Optional[str]
    birth_place: Optional[str]

# Yargy-rules

In [3]:
from dataclasses import dataclass
from typing import Optional
import json

from yargy import (
    Parser,
    rule,
    or_,
    and_,
    not_
)
from yargy.pipelines import (
    pipeline,
    morph_pipeline
)
from yargy.predicates import (
    eq,
    in_,
    type,
    normalized,
    dictionary,
    gte,
    lte,
    gram,
    is_capitalized
)
from yargy.interpretation import (
    fact,
    attribute
)
from yargy.tokenizer import MorphTokenizer, EOL
from yargy.relations import gnc_relation

from ipymarkup import show_span_ascii_markup as show_markup



@dataclass
class Entry:
    name: str
    birth_date: Optional[str]
    birth_place: Optional[str]

INT = type('INT')


#Name
Name = fact(
    "Name",
    ["first", "last"]
)

gnc = gnc_relation()

NAME = rule(
    gram("Name")
    .match(gnc)
    .interpretation(
        Name.first.inflected()
    ),
    gram("Surn")
    .match(gnc)
    # .optional()
    .interpretation(
        Name.last.inflected()
    )
).interpretation(
    Name
)

# Правила для даты рождения
Date = fact(
    'Date',
    ['year', 'month', 'day']
)

MONTHS = {
    'январь': 1,
    'февраль': 2,
    'март': 3,
    'апрель': 4,
    'май': 5,
    'июнь': 6,
    'июль': 7,
    'август': 8,
    'сентябрь': 9,
    'октябрь': 10,
    'ноябрь': 11,
    'декабрь': 12
}

MONTH_NAME = dictionary(
    MONTHS
).interpretation(
    Date.month.normalized().custom(MONTHS.get)
)

MONTH_NUM = and_(
    gte(1),
    lte(12)
).interpretation(
    Date.month.custom(int)
)

DAY = and_(
    gte(1),
    lte(31)
).interpretation(
    Date.day.custom(int)
)

YEAR = and_(
    gte(1900),
    lte(2100)
).interpretation(
    Date.year.custom(int)
)

DATE = rule(
    DAY,
    rule('.').optional(),
    or_(MONTH_NAME, MONTH_NUM),
    rule('.').optional(),
    YEAR,
    morph_pipeline(["год"]).optional()
).interpretation(
    Date
)


# Правила для места рождения
Place = fact(
    "Place",
    ("birth_place",)
)

PLACE = rule(
    morph_pipeline({'город', 'село', 'деревня', 'поселок'}).optional(),
    rule(is_capitalized().repeatable(max=2)).interpretation(
        Place.birth_place.normalized()
    )
).interpretation(
    Place
)

PersonFact = fact(
    'PersonFact',
    ('name', 'birth_date', 'birth_place')
)

# Полное правило для описания человека
PERSON = rule(
    or_(
        rule(
            NAME.interpretation(
                    PersonFact.name
                ),
            rule(
                morph_pipeline(['родиться']),
                rule('в').optional(),
                PLACE.optional().interpretation(
                        PersonFact.birth_place
                    ),
                DATE.optional().interpretation(
                        PersonFact.birth_date
                    ),
            ).optional(),
        ),
        rule(
            NAME.interpretation(
                    PersonFact.name
                ),
            rule(
                morph_pipeline(['родиться']),
                DATE.optional().interpretation(
                        PersonFact.birth_date
                    ),
                rule('в').optional(),
                PLACE.optional().interpretation(
                        PersonFact.birth_place
                    )
            ).optional()
        )
    )
).interpretation(
    PersonFact
)

parser = Parser(PERSON)


def extract_info(text):
    matches = []
    match = parser.findall(text)
    for entry in match:
        name = f'{entry.fact.name.first}{' '+entry.fact.name.last if entry.fact.name.last else ""}'
        birth_date = f'{entry.fact.birth_date.day}.{entry.fact.birth_date.month}.{entry.fact.birth_date.year}' if entry.fact.birth_date else None
        birth_place = entry.fact.birth_place.birth_place if entry.fact.birth_place else None
        matches.append(
            Entry(name=name, 
                  birth_date=birth_date,
                  birth_place=birth_place))
    return matches



In [4]:
from tqdm import tqdm

entries = []
for idx, row in tqdm(df.iterrows()):
    info = extract_info(row['text'])
    if info:
        entries.extend(info)


10000it [01:45, 94.84it/s]


In [5]:
[x for x in entries if x.birth_place]

[Entry(name='андрей курносенко', birth_date=None, birth_place='севастополь'),
 Entry(name='иосиф кобзон', birth_date=None, birth_place='час ярый'),
 Entry(name='яковлевюрий яковлев', birth_date=None, birth_place='москва'),
 Entry(name='николай караченцов', birth_date='27.10.1944', birth_place='москва'),
 Entry(name='инна лиснянская', birth_date=None, birth_place='баку'),
 Entry(name='татьяна самойлова', birth_date='4.5.1934', birth_place='ленинград'),
 Entry(name='владимир высоцкий', birth_date=None, birth_place='москва'),
 Entry(name='мэри дональдсон', birth_date=None, birth_place='хобарт'),
 Entry(name='борис васильев', birth_date='21.5.1924', birth_place='смоленск'),
 Entry(name='дэниел эдельман', birth_date=None, birth_place='нью'),
 Entry(name='зинаида серебрякова', birth_date='28.6.1913', birth_place='царский село'),
 Entry(name='игорь кваша', birth_date=None, birth_place='москва'),
 Entry(name='полина жеребцова', birth_date=None, birth_place='грозный'),
 Entry(name='алексей реми

In [238]:
[x for x in entries if x.birth_date]

[Entry(name='николай караченцов', birth_date='27.10.1944', birth_place='москва'),
 Entry(name='татьяна самойлова', birth_date='4.5.1934', birth_place='ленинград'),
 Entry(name='борис васильев', birth_date='21.5.1924', birth_place='смоленск'),
 Entry(name='лев дуров', birth_date='23.12.1931', birth_place=None),
 Entry(name='зинаида серебрякова', birth_date='28.6.1913', birth_place='царский село')]